# Sync with `wandb sync`

Usage
`wandb sync [OPTIONS] [PATH]...`

Ref: https://docs.wandb.ai/ref/cli/wandb-sync

| Option | Description |
| ------ | ----------- |
| --id | The run you want to upload to. |
| -p, --project | The project you want to upload to. |
| -e, --entity | The entity to scope to. |
| --sync-tensorboard / --no-sync-tensorboard | Stream tfevent files to wandb. |
| --include-globs | Comma seperated list of globs to include. |
| --exclude-globs | Comma seperated list of globs to exclude. |
| --include-online / --no-include-online | Include online runs |
| --include-offline / --no-include-offline | Include offline runs |
| --include-synced / --no-include-synced | Include synced runs |
| --mark-synced / --no-mark-synced | Mark runs as synced |
| --sync-all | Sync all runs |
| --clean | Delete synced runs |
| --clean-old-hours | Delete runs created before this many hours. To be used alongside --clean flag. |
| --clean-force | Clean without confirmation prompt. |
| --show | Number of runs to show |
| --help | Show this message and exit. |

## Resumming with `resume=True` to `wandb.init()`

You can have wandb automatically resume runs by passing `resume=True` to `wandb.init()`. If your process doesn't exit successfully, the next time you run it wandb will start logging from the last step.

Ref: https://docs.wandb.ai/guides/track/advanced/resuming

```python
import wandb
import torch
import torch.nn as nn
import torch.optim as optim

PROJECT_NAME = 'pytorch-resume-run'
CHECKPOINT_PATH = './checkpoint.tar'
N_EPOCHS = 100

# Dummy data
X = torch.randn(64, 8, requires_grad=True)
Y = torch.empty(64, 1).random_(2)
model = nn.Sequential(
    nn.Linear(8, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)
metric = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
epoch = 0
run = wandb.init(project=PROJECT_NAME, resume=True)
if wandb.run.resumed:
    checkpoint = torch.load(wandb.restore(CHECKPOINT_PATH))
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

model.train()
while epoch < N_EPOCHS:
    optimizer.zero_grad()
    output = model(X)
    loss = metric(output, Y)
    wandb.log({'loss': loss.item()}, step=epoch)
    loss.backward()
    optimizer.step()

    torch.save({ # Save our checkpoint loc
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, CHECKPOINT_PATH)
    wandb.save(CHECKPOINT_PATH) # saves checkpoint to wandb
    epoch += 1
```

## Syncinig Local Files with `wandb sync --sync-all`

### Syncing offline files


could you also add the flag `--include-offline` when syncing all offline runs? The cmd should be: `wandb sync --include-synced --include-offline --sync-all`. The reason we need to include this flag is that in offline mode all your runs have a prefix offline (you can check these out in the wandb dir or the zip file you shared above). 

Therefore, the code at this line suggests that if this flag is provided, all runs in offline-run-* will be included when syncing to W&B.

Ref: https://github.com/wandb/client/issues/3111

### Syncing previously synced files, where run-id is deleted on the cloud

unfortunately today you can not re-use deleted run ids in a project. We provide a helper method wandb.util.generate_id() which can be used to generate a random set of characters to append to your experiment name.

Here's a quick way to generate a new id: `python -c "import wandb; print(wandb.util.generate_id())"`
This way you can re-upload the data from your old run under a new ID by passing the --id argument to wandb sync. 

For ex: you can try to sync to a new run_id using `wandb sync --id new_run_id` `[PATH_TO_THE_OLD_RUN]`. And, in your case, it would be `wandb sync --id new_run_id ./wandb/run-3lwu1nrf`

Note that if you have an existing run that isn't deleted which you want to append data to you should see the resuming documentation.

Ref: https://github.com/wandb/client/issues/3111

Alternatively, you can do:

![Renaming existing run](imgs/resync.PNG)

* delete `run-dt4lbsyu.wandb.synced`
* generate `<new_run_id>` using `python -c "import wandb; print(wandb.util.generate_id())"`
* rename `run-dt4lbsyu.wandb.wandb` to `<new_run_id>`
* rename `run-20220605_110427-dt4lbsyu` to `run-20220605_110427-<new_run_id>`
* repeat the steps for all the runs in wandb folder, and run `wandb sync sync-all .` command
* or you can run `wandb sync --id <new_run_id> ./run-20220605_110427-<new_run_id>`

### Resyncing sycned files

* goto parent directory of wandb directory
* `wandb sync --include-synced --sync-all .` or
* `wandb sync --include-synced ./wandb/run-20220605_222139-<run_id>  `

## Working Offline

This to save your file locally without syncing. Later you can call `wandb sync` to sync your files.

* goto parent directory of wandb directory
* `wandb offline`

this will create a `settings` file inside `wandb` folder. Change the setting file or run 
the following command if you want to go online later.

`wandb online`

# `wandb.config`

## Update Config Files

You can use the public API to add values your config file, even after the run has finished.

    import wandb
    api = wandb.Api()
    run = api.run("username/project/run_id")
    run.config["foo"] = 32
    run.update()